In [2]:
import csv
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import mpld3
import matplotlib.pyplot as plt
import matplotlib as mpl
import requests
import time
import datetime
import warnings
warnings.simplefilter('ignore')
import pymystem3
mystem = pymystem3.Mystem()
from __future__ import unicode_literals
from nltk.util import ngrams
from collections import Counter
#nltk.download('stopwords')
#nltk.download('punkt')
import nltk
from nltk.corpus import brown

In [3]:
################################################################################################################################
table1 = pd.read_csv("./data_1.csv", encoding = 'utf-8',  sep=',') 
table2 = pd.read_csv("./data2.csv", encoding = 'utf-8',  sep=',') 
print(len(table1), len(table2))
table = pd.concat([table1, table2]) 
print(len(table))
table.drop_duplicates(keep = "first", inplace = True)
print(len(table))
table.reset_index(inplace = True, drop = True)
table = table.fillna('')

453 833
1286
736


In [3]:
table2.head()

owner_id                                               text  \
0    855757   Зачем подростку Бетховен История флейты. Обсу...   
1    856815   🎉🎊💥🎉🎊💥🎉🎊💥🎉🎊 Внимание, кто готов включиться в ...   
2    856815   🎉🎊💥🎉🎊💥🎉🎊💥🎉🎊 Внимание, кто готов включиться в ...   
3    856815   🎉🎊💥🎉🎊💥🎉🎊💥🎉🎊 Внимание, кто готов включиться в ...   
4    863442   Счастливая Джина! Друзья! Приходите на праздн...   

                                                name  \
0   Петровское (Приозерский район), ж/д Петяярви....   
1   Кинокультура Аудиокниги Типичный творческий П...   
2   Кинокультура Аудиокниги Типичный творческий П...   
3   Кинокультура Аудиокниги Типичный творческий П...   
4   KudaGo: Санкт-Петербург Журнал "Культурная Ст...   

                                         description     bdate  \
0   188732,  посёлок Петровское, Приозерский р-н,...      29.3   
1   Некоторые картины могут содержать сцены насил...  3.1.1982   
2   Некоторые картины могут содержать сцены насил...  3.1.1982   
3   Некоторые картины могут содержать сцены насил...  3.1.1982   
4   Проходите по нашим хештегам и легко находите ...       3.4   

              city              occupation                        status  \
0  Санкт-Петербург  РГПУ им. А. И. Герцена                           NaN   
1      Новосибирск                    НГПУ                           NaN   
2      Новосибирск                    НГПУ                           NaN   
3      Новосибирск                    НГПУ                           NaN   
4  Санкт-Петербург           Русский музей  Репетитор итальянского языка   

  about books education_form education_status  faculty faculty_name  \
0   NaN   NaN            NaN              NaN      NaN          NaN   
1   NaN   NaN            NaN              NaN      NaN          NaN   
2   NaN   NaN            NaN              NaN      NaN          NaN   
3   NaN   NaN            NaN              NaN      NaN          NaN   
4   NaN   NaN            NaN              NaN      NaN          NaN   

   graduation interests personal quotes  university university_name  
0         NaN       NaN      NaN    NaN         NaN             NaN  
1         NaN       NaN      NaN    NaN         NaN             NaN  
2         NaN       NaN      NaN    NaN         NaN             NaN  
3         NaN       NaN      NaN    NaN         NaN             NaN  
4         NaN       NaN      NaN    NaN         NaN             NaN

In [5]:
sel_col = table[['interests','city' ,'about', 'status', 'faculty_name', 'description']].copy()
stop_words=[]
stop_words_ru = nltk.corpus.stopwords.words('russian')
stop_words_en = nltk.corpus.stopwords.words('english')
f1 = codecs.open('ru-stop.txt', 'r','utf-8')
stop_words = f1.read()
stop_words = stop_words.replace('\r', ' ').replace('\n', '')
stop_words = stop_words.split()
stop_words.extend(stop_words_ru)
stop_words.extend(stop_words_en)
stop_words= sorted(stop_words)

In [6]:
sel_col['description'][:][0:1000]
print(sel_col['description'][1][0:1000])

In [7]:
sel_col.head(10)

interests             city about  \
0                                                     Санкт-Петербург         
1                                                     Санкт-Петербург         
2  Мода, музыка, искусство (живопись, фотография,...  Санкт-Петербург         
3                                                           Тель-Авив         
4                                                                Сочи         
5                                                     Санкт-Петербург         
6                                                              Москва         
7                                                     Санкт-Петербург         
8                                                              Москва         
9                                                     Санкт-Петербург         

                                              status  \
0                                                      
1                  "Моё гнездо-что хочу,то и делаю!"   
2                                                      
3                                                      
4                                                      
5  Найти дело всей жизни. Стать супер профессиона...   
6                                                      
7                                                      
8                                                      
9                                                      

                                        faculty_name  \
0                                                      
1                                                      
2  Институт наук о Земле (бывш. Факультет географ...   
3                                                      
4                                                      
5                                                      
6                                                      
7                                                      
8                                                      
9                                                      

                                         description  
0   Официальное сообщество российского общедоступ...  
1   Каждый день мы рассказываем о том, почему Пет...  
2   Дорогие друзья! Приглашаем Вас присоединиться...  
3      Проходите по нашим хештегам и легко находи...  
4   RT — это круглосуточный информационный телека...  
5   СММ ВКонтакте Все о менеджменте и управлении ...  
6   Все о психологии простым языком. Почта Mail.R...  
7   Онлайн кинотеатр Здравствуйте, дорогие друзья...  
8   История, политика и культура. Злые тексты, ор...  
9   Официальная страница финала Северного Евразий...

In [7]:
%%time
p = codecs.open('dict_gensim_736.txt','w', 'utf-8')
x=[]
y=[]
s=[]
a=[]

tt = ''
unit_col = sel_col
I = len(sel_col)
J = len (sel_col.columns)
for i in range (736):
    tt = ''
    for j in range (J):
        tt = tt + (unit_col.iloc[i, j])
    word = nltk.word_tokenize(tt) # токинезация текстов i-го пользователя 

    word_ws = [w.lower()  for w in word if w.isalpha() ] # исключение слов и символов      

    word_w2 = [w for w in word_ws if w not in (stop_words)  ] #нижний регистр 
    lem = mystem.lemmatize((" ").join(word_w2)) # лемматизация i-го документа
    lema = [w for w in lem if w.isalpha() and len(w)>1]
    lema = [w for w in lema if w not in (stop_words)  ] #нижний регистр 

    #freq = nltk.FreqDist(lema)
    #z = [] # обновление списка для нового документа
    #z = [(key + ":" ) for key,val in freq.items() if val>1] # частота упоминания через : от слова    
    #z1 = [(key) for key,val in freq.items() if val>1]
    
    #для bigartm
    #p.write("|text" +" "+(" ").join(z) + "\r\n") # запись в мешок слов с меткой |text   
    
    #для gensim
    p.write((" ").join(lema) + "\r\n") # запись в мешок слов
#     list1 = freq.elements()
#     counts = dict(Counter(list1).most_common())
#     terms = list(counts.keys())
#     freqs = list(counts.values())
    
#     a = freqs[0:15]
#     b = terms[0:15]# TOP-10 для частот a  и слов b в i -м документе 
    
#     x.append(i)#список номеров документов
#     y.append(len(freq))#список количества слов в документах  
    
#     y_pos = np.arange(1,len(a)+1, 1)#построение TOP-10 диаграмм      
#     performance = a
#     plt.barh(y_pos, a)
#     plt.yticks(y_pos, b)
#     plt.xlabel(u'Количество слов')
#     plt.title(u'Частоты слов у юзверя № %i'%i, size = 10)
#     plt.grid(True)
#     plt.show()  
#     a[:] = []
#     b[:] = [] 
# plt.title(u'Количество слов в документах', size = 10)
# plt.xlabel(u'Номера документов', size = 10)
# plt.ylabel(u'Количество слов', size = 10)
# plt.bar(x,y, 1)
# plt.grid(True)
# plt.show()
p.close()  

Wall time: 18min 47s


In [12]:
print I

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(I)? (<ipython-input-12-c722da17d3c6>, line 1)